## Baseline Models

This notebook builds and evaluates baseline deep learning models for crop yield prediction using the processed features from `02_feature_engineering.ipynb`.
The models include:

- Convolutional Neural Network (CNN)
- Long Short-Term Memory network (LSTM)
- Deep Neural Network (DNN)


# Ignore Future Warnings 

In [58]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Import Libraries 

In [59]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten


# Load Processed Data 

In [60]:
# Load features and targets
X = np.load("../processed/X_final.npy")
y = np.load("../processed/y.npy")        # original target
y_log = np.load("../processed/y_log.npy")  # log-transformed target

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (56717, 225)
y shape: (56717,)


# Train -- Test Split 

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (45373, 225)
Test shape: (11344, 225)


## Building CNN Model

We first reshape the training and test feature matrices to make them compatible with a 1D Convolutional Neural Network.


In [62]:
# Reshape for Conv1D: (samples, timesteps, features)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("CNN input shape:", X_train_cnn.shape)


CNN input shape: (45373, 225, 1)


### Define and Compile CNN Model

We create a simple baseline 1D CNN with two convolutional layers, followed by a flatten layer and dense layers for regression output.


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense

# Define CNN model
cnn_model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    Conv1D(filters=16, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # regression output
])

# Compile model
cnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print summary
cnn_model.summary()


c:\Users\pc-msi\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 223, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 221, 16)        │         1,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 3536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │       226,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 228,113 (891.07 KB)

 Trainable params: 228,113 (891.07 KB)

 Non-trainable params: 0 (0.00 B)

### Train the CNN Model

We train the CNN on the training data, using 20% of it for validation.  
For this baseline, we use 10 epochs and a batch size of 64.


In [64]:
##  Train CNN 

history = cnn_model.fit(
    X_train_cnn, y_train,         # training features and target
    validation_split=0.2,         # 20% of training data for validation
    epochs=10,                    # baseline epochs
    batch_size=64                 # standard batch size
)


Epoch 1/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 2.8897 - mae: 0.8855 - val_loss: 0.3832 - val_mae: 0.4547
Epoch 2/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.3186 - mae: 0.4224 - val_loss: 0.3683 - val_mae: 0.4378
Epoch 3/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - loss: 0.2993 - mae: 0.4072 - val_loss: 0.3048 - val_mae: 0.3925
Epoch 4/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - loss: 0.3084 - mae: 0.4130 - val_loss: 0.3340 - val_mae: 0.4268
Epoch 5/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - loss: 0.3031 - mae: 0.4100 - val_loss: 0.3382 - val_mae: 0.4309
Epoch 6/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.3020 - mae: 0.4100 - val_loss: 0.3342 - val_mae: 0.4087
Epoch 7/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.3034 - mae: 0.4099 - val_loss: 0.3510 - val_mae: 0.4264
Epoch 8/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3076 - mae: 0.4140 - val_loss: 0.3203 - val_mae: 0.4151
Epoch 9/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/ste

In [65]:
## Evaluate on Test Set

test_loss, test_mae = cnn_model.evaluate(X_test_cnn, y_test)
print("Test MSE:", test_loss)
print("Test MAE:", test_mae)


355/355 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3881 - mae: 0.4639
Test MSE: 0.38806986808776855
Test MAE: 0.46392112970352173


In [66]:
## Save the model 

# cnn_model.save("../processed/cnn_baseline_model.h5")
cnn_model.save("../processed/cnn_baseline_model.keras")



## Baseline LSTM Model

We build a simple LSTM network for crop yield prediction, using the processed features.


In [67]:
# Reshape for LSTM
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("LSTM input shape:", X_train_lstm.shape)


LSTM input shape: (45373, 225, 1)


# LSTM model 

In [68]:
from tensorflow.keras.layers import Input

lstm_model = Sequential([
    Input(shape=(X_train_lstm.shape[1], 1)),  # input layer
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1)  # regression output
])

lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
lstm_model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,009 (74.25 KB)

 Trainable params: 19,009 (74.25 KB)

 Non-trainable params: 0 (0.00 B)

# Train LSTM

In [69]:
history_lstm = lstm_model.fit(
    X_train_lstm, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64
)


Epoch 1/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 44s 75ms/step - loss: 5.0953 - mae: 1.3060 - val_loss: 1.2078 - val_mae: 0.9037
Epoch 2/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 41s 72ms/step - loss: 1.1796 - mae: 0.9007 - val_loss: 1.1777 - val_mae: 0.8904
Epoch 3/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 54s 95ms/step - loss: 1.1476 - mae: 0.8823 - val_loss: 1.1113 - val_mae: 0.8562
Epoch 4/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 33s 57ms/step - loss: 1.0847 - mae: 0.8461 - val_loss: 1.0680 - val_mae: 0.8321
Epoch 5/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 31s 54ms/step - loss: 1.0307 - mae: 0.8198 - val_loss: 1.0287 - val_mae: 0.8176
Epoch 6/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 32s 56ms/step - loss: 0.8827 - mae: 0.7519 - val_loss: 0.7698 - val_mae: 0.6939
Epoch 7/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - loss: 0.6559 - mae: 0.6402 - val_loss: 0.6276 - val_mae: 0.6126
Epoch 8/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 33s 59ms/step - loss: 0.5759 - mae: 0.5983 - val_loss: 0.5590 - val_mae: 0.5796
Epoch 9/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 

# Evaluate LSTM

In [70]:
test_loss_lstm, test_mae_lstm = lstm_model.evaluate(X_test_lstm, y_test)
print("LSTM Test MSE:", test_loss_lstm)
print("LSTM Test MAE:", test_mae_lstm)


355/355 ━━━━━━━━━━━━━━━━━━━━ 25s 72ms/step - loss: 0.5457 - mae: 0.5769
LSTM Test MSE: 0.5456959009170532
LSTM Test MAE: 0.5768837928771973


In [71]:
## Save LSTM model
lstm_model.save("../processed/lstm_baseline_model.keras")


## Baseline DNN Model

We build a simple fully connected Deep Neural Network for crop yield prediction using the processed features.


## Define DNN model 

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

dnn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # regression output
])

dnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
dnn_model.summary()


c:\Users\pc-msi\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 128)            │        28,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,297 (153.50 KB)

 Trainable params: 39,297 (153.50 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
## Train DNN 

history_dnn = dnn_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64
)


Epoch 1/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - loss: 6.3292 - mae: 1.1769 - val_loss: 0.2837 - val_mae: 0.3793
Epoch 2/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.2020 - mae: 0.3260 - val_loss: 0.1815 - val_mae: 0.2913
Epoch 3/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 0.1315 - mae: 0.2550 - val_loss: 0.1358 - val_mae: 0.2329
Epoch 4/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.1069 - mae: 0.2274 - val_loss: 0.1248 - val_mae: 0.2204
Epoch 5/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 0.0959 - mae: 0.2145 - val_loss: 0.1217 - val_mae: 0.2184
Epoch 6/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.0911 - mae: 0.2079 - val_loss: 0.1193 - val_mae: 0.2163
Epoch 7/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 0.0881 - mae: 0.2055 - val_loss: 0.1102 - val_mae: 0.2049
Epoch 8/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.0864 - mae: 0.2029 - val_loss: 0.1274 - val_mae: 0.2393
Epoch 9/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 4s 6

In [74]:
## Evaluate DNN 
test_loss_dnn, test_mae_dnn = dnn_model.evaluate(X_test, y_test)
print("DNN Test MSE:", test_loss_dnn)
print("DNN Test MAE:", test_mae_dnn)


355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0943 - mae: 0.2029
DNN Test MSE: 0.09427804499864578
DNN Test MAE: 0.20291753113269806


In [75]:
## Save DNN model 

dnn_model.save("../processed/dnn_baseline_model.keras")
